In [1]:
import numpy as np
import pandas as pd
import torch
import os
import re

In [2]:
from transformers import AutoModel, AutoTokenizer

In [3]:
# model_name = 'DeepPavlov/rubert-base-cased-sentence'
model_name = 'DeepPavlov/bert-base-multilingual-cased-sentence'
# model_name = 'DeepPavlov/distilrubert-small-cased-conversational'
# model_name = 'DeepPavlov/distilrubert-tiny-cased-conversational'
model = AutoModel.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [4]:
def get_first_header(note):
    if '## ' in note:
        first_header_prefix = '## '
        if '### ' in note:
            first_header_prefix = '### '

        first_header = note.split(first_header_prefix)[1].split('\n')[0]
    else:
        first_header = ''

    return first_header

def clean(note):
    # remove zero-links
    note = re.sub(r'\[.*\]', '', note)

    # remove tags and headers
    note = re.sub(r'\#.*\n', '', note)

    # remove \n
    note = re.sub('\n', ' ', note)

    # remove lines
    note = re.sub('---', ' ', note)

    # remove **
    note = re.sub('\*', '', note)
    
    return note

In [5]:
length_thr = 20
# device = 'cuda'
device = 'cpu'
encode_kwargs = {'truncation': True, 'padding': 'max_length', 'pad_to_multiple_of': 1, 'max_length':512}

In [6]:
db_path = '/home/booydar/Documents/Sync/obsidian-db/'


def parse_folder(path):
    # print(f'Parsing {path}')

    path, folders, files = next(os.walk(path))

    db_df = pd.DataFrame()
    if len(folders) > 0:
        for f in folders:
            folder_path = os.path.join(path, f)
            f_res_df = parse_folder(folder_path)
            db_df = pd.concat([db_df, f_res_df])

    for fn in files:
        if '.md' not in fn:
            continue

        filepath = os.path.join(path, fn)
        with open(filepath, 'r') as f:
            note = f.read()

        if len(note) < length_thr:
            continue

        header = get_first_header(note)
        if not header: 
            header = fn[:-3]

        cleaned_note = clean(note)
        
        tokenized_header = tokenizer.encode(header, **encode_kwargs)
        tokenized_note = tokenizer.encode(cleaned_note, **encode_kwargs)

        note_dict = {'name': fn, 'path':filepath, 'header': header, 'note': cleaned_note, 'tokenized_header':[tokenized_header], 'tokenized_note':[tokenized_note]}

        db_df = pd.concat([db_df, pd.DataFrame(note_dict)])
    
    return db_df


In [10]:
db_df = parse_folder(db_path)
# db_df.to_csv('tables/db_df.csv', index = False)
# db_df = pd.read_csv('tables/db_df.csv')

In [11]:
model.to(device)
tokenized_headers = torch.Tensor(np.vstack(db_df.tokenized_header.values)).long()#.to(device)
tokenized_notes = torch.Tensor(np.vstack(db_df.tokenized_note.values)).long()#.to(device)

# vectorized_headers = model(tokenized_headers)
# vectorized_notes = model(tokenized_notes)

In [20]:
def process_by_batch(model, tensor, batch_size = 2):
    n_chunks = tensor.shape[0] // batch_size
    chunked = torch.chunk(tensor, n_chunks)

    outputs = []
    for batch in chunked:
        out = model(batch.to(device)).last_hidden_state[:, 0, :]
        outputs.append(out.cpu().detach())
    return torch.cat(outputs)

In [21]:
batch_size = 2
vectorized_headers = process_by_batch(model, tokenized_headers, batch_size)

In [22]:
vectorized_notes = process_by_batch(model, tokenized_notes, batch_size)

In [ ]:
num_notes = db_df.shape[0]
# header_embeddings = vectorized_headers.last_hidden_state.reshape(num_notes, -1)
# header_embeddings = vectorized_headers.last_hidden_state.mean(dim=-2)
header_embeddings = vectorized_headers.last_hidden_state[:, 0, :].detach().numpy()
note_embeddings = vectorized_notes.last_hidden_state[:, 0, :].detach().numpy()

In [ ]:
from sklearn.cluster import KMeans

### headers

In [ ]:
cluster = KMeans(n_clusters=10)
cluster = cluster.fit(header_embeddings)
header_clusters = cluster.predict(header_embeddings)
headers = db_df.header.values

In [ ]:
for c in np.unique(header_clusters):
    group = headers[header_clusters == c]
    print(f'Cluster {c}\n{group}\n\n')

#### notes

In [ ]:
notes = db_df.note.values

In [ ]:
cluster = KMeans(n_clusters=10)
cluster = cluster.fit(note_embeddings)
note_clusters = cluster.predict(note_embeddings)

for c in np.unique(note_clusters):
    group = notes[note_clusters == c]
    print(f'Cluster {c}\n{group}\n\n')